In [1]:
#!git clone https://github.com/RUCAIBox/HaluEval.git

In [2]:
!git submodule update --recursive

In [3]:
# !pip install transformers[torch]

In [4]:
import pickle
# import graphvite as gv
#import dataset
import nltk
import re
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/hice1/zaristei3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hice1/zaristei3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/hice1/zaristei3/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/hice1/zaristei3/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hice1/zaristei3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# !pip install transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [6]:
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login

notebook_login()

In [7]:
#!pip install datasets

In [8]:
#!wget -O ~/scratch/processed_data.pkl https://www.dropbox.com/scl/fi/vp5c9hcyqnaottfre5yas/processed_data.pkl?rlkey=i0gko0chfwyj5fn4rhsx9o3ve&dl=0

In [9]:
import os

TRANSFORMERS_CACHE = os.path.expanduser("~/scratch/transformers_cache")
os.makedirs(TRANSFORMERS_CACHE, exist_ok=True)

HF_DATASETS_CACHE = os.path.expanduser("~/scratch/hf_datasets_cache")
os.makedirs(HF_DATASETS_CACHE, exist_ok=True)

In [10]:
processed_file = open(os.path.expanduser("~/scratch/processed_data.pkl"), "rb")
graph_data = pickle.load(processed_file)

In [11]:
import json

# Opening JSON file
data_list = []
with open('submodules/HaluEval/data/qa_data.json') as json_file:
    for jsonObj in json_file:
        data_dict = json.loads(jsonObj)
        data_list.append(data_dict)



In [12]:
connection_dict = {obj["question"].strip().lower() : i for i, obj in enumerate(data_list)}

In [13]:
len(connection_dict)

10000

In [14]:
from datasets import load_dataset

def get_data(data_path):

  ds = load_dataset("json", data_files=data_path, split="train", cache_dir=HF_DATASETS_CACHE)
  ds = ds.train_test_split(test_size=0.2)
  return ds

#print(os.listdir())

ds = get_data("submodules/HaluEval/data/qa_data.json")
print(ds)

Found cached dataset json (/home/hice1/zaristei3/scratch/hf_datasets_cache/json/default-6f3eceb850a0e4ec/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


DatasetDict({
    train: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['knowledge', 'question', 'right_answer', 'hallucinated_answer'],
        num_rows: 2000
    })
})


In [15]:
import torch

def preprocess_function(examples):
  questions = [q.strip().lower() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      [c.lower() for c in examples["knowledge"]],
      max_length=384,
      truncation="only_second",
      return_offsets_mapping=True,
      padding="max_length",
  )
  offset_mapping = inputs.pop("offset_mapping")
  answers = examples["right_answer"]
  start_positions = []
  end_positions = []
  graph_embeddings = []

  for i, offset in enumerate(offset_mapping):
      answer = answers[i]
      start_char = examples["knowledge"][i].find(answer)
      end_char = start_char + len(answer)
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

      corr_index = connection_dict[questions[i]]
      graph_index = graph_data[corr_index]
      embeddings = torch.zeros(384, 512)
      token_list = inputs.input_ids[i]
      for ent, embedding in graph_index["knowledge"].items():
          ent_token_list = [tokenizer(ent).input_ids[1:-1], tokenizer(" " + ent).input_ids[1:-1]]

          for i in range(len(token_list) - len(list(ent_token_list)[0])):
            if token_list[i:i+len(list(ent_token_list)[0])] in ent_token_list:
              embeddings[i:i+len(list(ent_token_list)[0]), :] = torch.broadcast_to(torch.from_numpy(embedding), (len(list(ent_token_list)[0]), 512))
      for ent, embedding in graph_index["question"].items():
          ent_token_list = [tokenizer(ent).input_ids[1:-1], tokenizer(" " + ent).input_ids[1:-1]]

          for i in range(len(token_list) - len(list(ent_token_list)[0])):
            if token_list[i:i+len(list(ent_token_list)[0])] in ent_token_list:
              embeddings[i:i+len(list(ent_token_list)[0]), :] = torch.broadcast_to(torch.from_numpy(embedding), (len(list(ent_token_list)[0]), 512))
      graph_embeddings.append(embeddings)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  inputs["graph_embeddings"] = graph_embeddings

  return inputs

In [16]:
ds["train"][1]

{'knowledge': 'Leigh Silverman is an American director for the stage, both Off-Broadway and on Broadway. She was nominated for the 2014 Tony Award, Best Direction of a Musical for the musical "Violet" and the 2008 Drama Desk Award, Outstanding Director of a Play for the play "From Up Here".Violet is a musical with music by Jeanine Tesori and libretto by Brian Crawley based on the short story "The Ugliest Pilgrim" by Doris Betts.',
 'question': 'Leigh Silverman was nominated for the 2014 Tony Award, Best Direction for a musical based on which short story ?',
 'right_answer': 'The Ugliest Pilgrim',
 'hallucinated_answer': 'Leigh Silverman\'s musical "Violet" was based on an original story.'}

In [17]:
print(ds["train"][1]["question"].strip())
corr_index = connection_dict[ds["train"][1]["question"].strip().lower()]
print(corr_index)
#print(graph_data[corr_index])
graph_index = graph_data[corr_index]
#for ent, embedding in graph_index["knowledge"].items():
#    graph_embeddings.append(embedding)
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)

for k, v in graph_index["question"].items():
  print("{}, {}, {}".format(k, tokenizer(k).input_ids[1:-1], tokenizer(" " + k).input_ids[1:-1]))
for k, v in graph_index["knowledge"].items():
  print("{}, {}, {}".format(k, tokenizer(k).input_ids[1:-1], tokenizer(" " + k).input_ids[1:-1]))
print(tokenizer("Teen Wolf"))
print(str(tokenizer.decode([664, 2143, 417])) + "," + str(tokenizer.decode([39556])))

Leigh Silverman was nominated for the 2014 Tony Award, Best Direction for a musical based on which short story ?
804
leigh silverman, [19343, 4334, 397], [2084, 8774, 4334, 397]
leigh silverman, [19343, 4334, 397], [2084, 8774, 4334, 397]
american, [8015, 12657], [38187, 260]
off-broadway, [1529, 12, 32990, 1970], [160, 12, 32990, 1970]
broadway, [32990, 1970], [4007, 1970]
violet, [705, 41790], [43973]
2008, [27418], [2266]
from up here", [7761, 62, 259, 113], [31, 62, 259, 113]
jeanine tesori, [267, 12001, 833, 326, 293, 6249], [1236, 12001, 833, 326, 293, 6249]
brian crawley, [428, 10713, 36526, 607], [741, 10713, 36526, 607]
the ugliest pilgrim, [627, 1717, 571, 27911, 43573, 757], [5, 1717, 571, 27911, 43573, 757]
doris betts, [417, 26787, 5673, 1872], [385, 26787, 5673, 1872]
{'input_ids': [0, 39556, 7602, 2], 'attention_mask': [1, 1, 1, 1]}
 young boyd,Teen


In [18]:
example = {
    "knowledge": [ds["train"][1]["knowledge"]],
    "question": [ds["train"][1]["question"]],
    "right_answer": [ds["train"][1]["right_answer"]],
    "hallucinated_answer": [ds["train"][1]["hallucinated_answer"]]
}
#print(clean_text(example))
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)
inputs = preprocess_function(example)
print(inputs)
print(tokenizer.decode(inputs["input_ids"][0]))

{'input_ids': [[0, 19343, 4334, 397, 21, 7076, 13, 5, 777, 326, 6119, 2354, 6, 275, 2698, 13, 10, 4388, 716, 15, 61, 765, 527, 17487, 2, 2, 19343, 4334, 397, 16, 41, 38187, 260, 736, 13, 5, 1289, 6, 258, 160, 12, 32990, 1970, 8, 15, 4007, 1970, 4, 79, 21, 7076, 13, 5, 777, 326, 6119, 2354, 6, 275, 2698, 9, 10, 4388, 13, 5, 4388, 22, 705, 41790, 113, 8, 5, 2266, 4149, 8429, 2354, 6, 3973, 736, 9, 10, 310, 13, 5, 310, 22, 7761, 62, 259, 845, 705, 41790, 16, 10, 4388, 19, 930, 30, 1236, 12001, 833, 326, 293, 6249, 8, 21748, 18235, 139, 30, 741, 10713, 36526, 607, 716, 15, 5, 765, 527, 22, 627, 1717, 571, 27911, 43573, 757, 113, 30, 385, 26787, 5673, 1872, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [19]:
torch.device('cuda')

device(type='cuda')

In [20]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)
tokenized_ds = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hice1/zaristei3/.conda/envs/kg_trainer/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_207234/2997175811.py", line 3, in <module>
    tokenized_ds = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hice1/zaristei3/.conda/envs/kg_trainer/lib/python3.11/site-packages/datasets/dataset_dict.py", line 851, in map
    {
  File "/home/hice1/zaristei3/.conda/envs/kg_trainer/lib/python3.11/site-packages/datasets/dataset_dict.py", line 852, in <dictcomp>
    k: dataset.map(
       ^^^^^^^^^^^^
  File "/home/hice1/zaristei3/.conda/envs/kg_trainer/lib/python3.11/site-packages/datasets/arrow_dataset.py", line 578, in wrapper
    out: Union["Dataset", "DatasetDict"] = func(self, *args, **kw

In [ ]:
#print(tokenized_ds["train"][0])
print(tokenized_ds["train"][0]["graph_embeddings"])
print(len(tokenized_ds["train"][0]["graph_embeddings"]))
print(len(tokenized_ds["train"][0]["graph_embeddings"][0]))
# print(tokenized_ds["train"]["end_positions"][0:10])

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import QuestionAnsweringModelOutput

class GraphEnrichedRoberta(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.llm_embedding_size = 768
    self.graph_embedding_size = 512
    self.llm = AutoModelForQuestionAnswering.from_pretrained(model_name)
    self.fc = nn.Linear(self.graph_embedding_size + self.llm_embedding_size, self.llm_embedding_size)

    for param in self.llm.roberta.encoder.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, start_positions, end_positions, graph_embeddings):
     outputs = self.llm.roberta(input_ids, attention_mask)
     sequence_outputs = outputs.last_hidden_state

     enriched_encodings = self.fc(torch.cat((sequence_outputs, graph_embeddings), 2))
     enriched_encodings = F.relu(enriched_encodings)

     logits = self.llm.qa_outputs(enriched_encodings)
     start_logits, end_logits = logits.split(1, dim=-1)
     start_logits = start_logits.squeeze(-1).contiguous()
     end_logits = end_logits.squeeze(-1).contiguous()

     total_loss = None
     if start_positions is not None and end_positions is not None:
          # If we are on multi-GPU, split add a dimension
         if len(start_positions.size()) > 1:
             start_positions = start_positions.squeeze(-1)
         if len(end_positions.size()) > 1:
             end_positions = end_positions.squeeze(-1)
          # sometimes the start/end positions are outside our model inputs, we ignore these terms
         ignored_index = start_logits.size(1)
         start_positions = start_positions.clamp(0, ignored_index)
         end_positions = end_positions.clamp(0, ignored_index)

         loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
         start_loss = loss_fct(start_logits, start_positions)
         end_loss = loss_fct(end_logits, end_positions)
         total_loss = (start_loss + end_loss) / 2

     return QuestionAnsweringModelOutput(
         loss=total_loss,
         start_logits=start_logits,
         end_logits=end_logits,
         hidden_states=outputs.hidden_states,
         attentions=outputs.attentions,
     )

In [ ]:
model = GraphEnrichedRoberta(model_name)

In [ ]:
with torch.no_grad():
  input_ids_test = torch.tensor(tokenized_ds["train"][0]["input_ids"]).unsqueeze(0)
  attn_mask_test = torch.tensor(tokenized_ds["train"][0]["attention_mask"]).unsqueeze(0)
  start_positions_test = torch.tensor(tokenized_ds["train"][0]["start_positions"]).unsqueeze(0)
  end_positions_test = torch.tensor(tokenized_ds["train"][0]["end_positions"]).unsqueeze(0)
  graph_embeddings_test = torch.tensor(tokenized_ds["train"][0]["graph_embeddings"]).unsqueeze(0)
  print(model(input_ids_test, attn_mask_test, start_positions_test, end_positions_test, graph_embeddings_test))

QuestionAnsweringModelOutput(loss=tensor(5.9746), start_logits=tensor([[ 1.2732e-01, -4.2468e-03,  8.5093e-02, -1.7349e-02, -1.1381e-02,
          1.7205e-02,  5.2489e-05, -2.2071e-02,  1.1532e-02, -4.7795e-02,
          3.8199e-02,  2.3437e-02,  3.8594e-03,  4.0029e-03,  5.1752e-02,
          4.7071e-02,  1.6280e-02, -8.5421e-04,  7.9077e-04, -2.9250e-02,
          2.0945e-02,  6.1799e-02,  8.3100e-02,  3.8242e-02,  1.4995e-01,
          1.2294e-01,  1.6828e-01,  1.1685e-02, -3.3141e-02,  6.9731e-02,
          8.4796e-02,  9.0301e-02,  8.4008e-03,  8.0968e-02,  7.1955e-03,
          3.5837e-02,  1.1836e-01,  1.7149e-02, -1.8052e-03, -7.6675e-04,
          5.0216e-02, -2.2695e-02,  3.9497e-02,  1.0423e-02,  9.0101e-02,
         -3.9284e-03,  5.4917e-02, -3.1455e-03,  1.6174e-02,  9.5413e-02,
         -8.2857e-02, -6.9017e-03, -2.2094e-02,  2.1134e-02,  2.4115e-02,
         -7.2224e-02, -1.6914e-02, -3.9402e-02,  1.0029e-02, -1.3669e-01,
          1.3960e-02,  5.5106e-02, -3.4658e-02, -

In [ ]:
training_args = TrainingArguments(
    output_dir="roberta_qa_model_with_graph",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.072100,No log
2,1.586100,No log


Epoch,Training Loss,Validation Loss
1,2.072100,No log
2,1.586100,No log
3,1.510300,No log


TrainOutput(global_step=1500, training_loss=1.7228113606770834, metrics={'train_runtime': 10196.5166, 'train_samples_per_second': 2.354, 'train_steps_per_second': 0.147, 'total_flos': 0.0, 'train_loss': 1.7228113606770834, 'epoch': 3.0})

In [ ]:
torch.save(model.state_dict(), 'graph_model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('graph_model_weights.pth'))

<All keys matched successfully>

In [ ]:
model

GraphEnrichedRoberta(
  (llm): RobertaForQuestionAnswering(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): 

In [ ]:
#tokenized_ds["test"]

predictions = trainer.predict(tokenized_ds["test"].remove_columns(["graph_embeddings"]))

In [ ]:
import numpy as np

for i in range(len(predictions.predictions)):
  print(predictions.predictions[i].shape)

print(predictions.predictions[0])

print(np.array(tokenized_ds["test"]["start_positions"]).shape)


(125,)
(2000, 384)
(2000, 384)
[1.0641084  1.6659701  1.9759023  1.2464397  1.970095   1.2182294
 1.7953851  1.3778882  1.089099   2.3120296  1.0138576  1.7308497
 1.5308025  1.2966044  1.4876668  0.62224865 2.2005458  1.19306
 1.9669992  1.8536528  2.293821   1.792011   0.7256372  1.629729
 2.1631832  1.4701074  1.4578855  1.6019956  1.2260633  1.738751
 2.4187698  1.3923588  1.9579014  1.3781996  2.456057   1.0412458
 1.7440045  1.8541543  2.8645754  0.93838847 1.7694396  1.6841547
 1.9321573  1.4411316  1.38648    1.194206   0.81366175 1.0065814
 0.8059305  1.9596742  0.79150534 1.9571393  0.9703902  1.6299062
 1.890945   1.146168   1.0196122  1.1374956  2.0560458  1.7159839
 1.968475   1.3967826  1.6296978  1.5666738  1.2158916  1.7590964
 2.4377935  1.9311898  1.9091766  2.4590755  1.9080064  1.0684565
 1.7132566  2.3912418  1.6143143  1.9749919  1.1747274  1.9486256
 1.5132129  1.6976893  1.2747147  1.1367294  2.944383   1.1048306
 1.9857776  1.3243711  1.0923741  1.5487192  1.53

In [ ]:
avg_loss = np.mean(predictions.predictions[0])
avg_loss

1.6121981

In [ ]:
start_logits = predictions.predictions[1]
end_logits = predictions.predictions[2]

start_positions = np.array(tokenized_ds["test"]["start_positions"])
end_positions = np.array(tokenized_ds["test"]["end_positions"])

In [ ]:
start_positions_pred = np.argmax(start_logits, axis=1)
end_positions_pred = np.argmax(end_logits, axis=1)

In [ ]:
print(start_positions_pred)
print(start_positions)
print(end_positions_pred)
print(end_positions)
print(np.sum(start_positions_pred == start_positions))
overlap = np.maximum(0.0, np.minimum(end_positions_pred, end_positions) - np.maximum(start_positions_pred, start_positions) + 1)
print(overlap)
val1 = np.abs(1.0 * overlap / (end_positions - start_positions + 1))
val2 = np.abs(1.0 * overlap / (end_positions_pred - start_positions_pred + 1))
print(np.min(val1))
print(np.max(val1))
print(np.min(val2))
print(np.max(val2))

[ 39  55  97 ... 127  51  24]
[ 39  55  97 ... 127  51  24]
[ 42  57 102 ... 130  56  29]
[ 42  57 102 ... 130  56  29]
1165
[4. 3. 6. ... 4. 6. 6.]
0.0
1.0
0.0
1.0


In [ ]:
def exact_match(start_logits, end_logits, start_positions, end_positions):
  start_positions_pred = np.argmax(start_logits, axis=1)
  end_positions_pred = np.argmax(end_logits, axis=1)
  same_start = start_positions_pred == start_positions
  same_end = end_positions_pred == end_positions
  return np.sum(np.logical_and(same_start, same_end)) / start_positions.shape[0]

In [ ]:
exact_match(start_logits, end_logits, start_positions, end_positions)

0.505

In [ ]:
def f1_score(start_logits, end_logits, start_positions, end_positions):
  start_positions_pred = np.argmax(start_logits, axis=1)
  end_positions_pred = np.argmax(end_logits, axis=1)
  overlap = np.maximum(0, np.minimum(end_positions_pred, end_positions) - np.maximum(start_positions_pred, start_positions) + 1)
  recall = np.abs(1.0 * overlap / (end_positions - start_positions + 1))
  precision = np.abs(1.0 * overlap / (end_positions_pred - start_positions_pred + 1))
  f1 = (2 * precision * recall) / (precision + recall + 0.00000001)
  return np.mean(f1)

In [ ]:
f1_score(start_logits, end_logits, start_positions, end_positions)

0.6002186955899961

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class GraphEnrichedRoberta(nn.Module):
  def __init__(self, model_name):
    super().__init__()
    self.llm_embedding_size = 768
    self.llm = AutoModelForQuestionAnswering.from_pretrained(model_name)

    for param in self.llm.roberta.encoder.parameters():
      param.requires_grad = False

  def forward(self, input_ids, attention_mask, start_positions, end_positions):
    return self.llm(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)

In [ ]:
model = GraphEnrichedRoberta(model_name)

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>